In [1]:
%pip install geopy

  Obtaining dependency information for geopy from https://files.pythonhosted.org/packages/e5/15/cf2a69ade4b194aa524ac75112d5caac37414b20a3a03e6865dfe0bd1539/geopy-2.4.1-py3-none-any.whl.metadata
  Obtaining dependency information for geographiclib<3,>=1.52 from https://files.pythonhosted.org/packages/9f/5a/a26132406f1f40cf51ea349a5f11b0a46cec02a2031ff82e391c2537247a/geographiclib-2.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   --- ------------------------------------ 10.2/125.4 kB ? eta -:--:--
   ------------ -------------------------- 41.0/125.4 kB 653.6 kB/s eta 0:00:01
   ---------------------------------------- 125.4/125.4 kB 1.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/40.3 kB ? eta -:--:--
   ---------------------------------------- 40.3/40.3 kB 1.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import time

In [4]:
from geopy.geocoders import Nominatim

# List of cities and their corresponding regions
cities = [
    "lisowo"
]

# Initialize geolocator
geolocator = Nominatim(user_agent="poland_cities_locator")

# Get coordinates for each city
city_coordinates = []
for city in cities:
    location = geolocator.geocode(city)
    if location:
        city_coordinates.append({
            "city": city,
            "latitude": location.latitude,
            "longitude": location.longitude
        })
    else:
        city_coordinates.append({
            "city": city,
            "latitude": None,
            "longitude": None
        })

import pandas as pd

# Convert to DataFrame for better representation
df_cities = pd.DataFrame(city_coordinates)
df_cities

,city,latitude,longitude
0,lisowo,52.463598,22.625987


In [5]:
file_path = '215d3c1ea7d0c562774a8c6e4b157ac0.xlsx'
data = pd.ExcelFile(file_path)
df = data.parse('2022')

In [11]:
columns_to_extract = [
   'Miejscowość',
   'Powiat',
   'Województwo'
]

df = df[columns_to_extract]


for index, entry in df.iterrows():
    # Construct the query string with additional parameters
    query = f"{entry['Miejscowość']}, {entry['Powiat']}, {entry['Województwo']}"
    
    try:
        # Use geocode with country codes to limit the search area
        location = geolocator.geocode(query, addressdetails=True, country_codes='pl', timeout=10)
        
        if location:
            city_coordinates.append({
                "City": entry['Miejscowość'],
                "Region": entry['Powiat'],
                "Latitude": location.latitude,
                "Longitude": location.longitude
            })
        else:
            city_coordinates.append({
                "City": entry['Miejscowość'],
                "Region": entry['Powiat'],
                "Latitude": None,
                "Longitude": None
            })
    except Exception as e:
        print(f"Error geocoding {entry['Miejscowość']}: {e}")
        city_coordinates.append({
            "City": entry['Miejscowość'],
            "Region": entry['Powiat'],
            "Latitude": None,
            "Longitude": None
        })
    
    # Respect Nominatim's usage policy by adding a delay between requests
    time.sleep(1)

# Convert the list to a DataFrame
df_cities = pd.DataFrame(city_coordinates)



In [13]:
df_copy = df_cities.copy()

In [18]:
columns_to_remove = ['city', 'latitude', 'longitude']
df_copy = df_copy.drop(columns=columns_to_remove)
df_copy = df_copy.drop(index=0)

In [19]:
df_copy

,City,Region,Latitude,Longitude
1,Świnoujście,Świnoujście,53.904445,14.266023
2,Świnoujście,Świnoujście,53.904445,14.266023
3,Świnoujście,Świnoujście,53.904445,14.266023
4,Świnoujście,Świnoujście,53.904445,14.266023
5,Świnoujście,Świnoujście,53.904445,14.266023
...,...,...,...,...
1402,Jaworzynka,cieszyński,49.527901,18.859085
1403,Małe Ciche,tatrzański,49.303148,20.072864
1404,Sarniki,braniewski,54.417277,20.017651
1405,Robity,bartoszycki,54.390181,20.365526


In [20]:
num_records_with_nan = df_copy.isna().any(axis=1).sum()

In [21]:
num_records_with_nan

0